In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import alignment_utils as au # Import our new module

# Load images in color and grayscale
img1_color = cv.imread('images/0_00985.jpg')
img1_gray = cv.cvtColor(img1_color, cv.COLOR_BGR2GRAY)

img2_color = cv.imread('images/1_00969.jpg')
img2_gray = cv.cvtColor(img2_color, cv.COLOR_BGR2GRAY)

In [ ]:
# Feature matching parameters
MATCH_METHOD = 'SIFT' # Can be 'SIFT' or 'ORB'
MIN_MATCH_COUNT = 10
LOWE_RATIO = 0.7

# Variable initialization
kp1, des1 = None, None
kp2, des2 = None, None
good_matches = []

In [ ]:
kp1, des1, kp2, des2, good_matches = au.detect_and_match(
    img1_gray, img2_gray, method=MATCH_METHOD, lowe_ratio=LOWE_RATIO
)
print(f"Found {len(good_matches)} good matches using {MATCH_METHOD}")

In [ ]:
M, mask = au.compute_homography(kp1, kp2, good_matches, MIN_MATCH_COUNT)

dst = None
if M is not None:
    # Calculate projection for visualization preparation
    # We'll calculate strictly the display points here, metrics in next block
    h, w = img1_gray.shape
    pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
    dst = cv.perspectiveTransform(pts, M)
else:
    print(f"Not enough matches are found - {len(good_matches)}/{MIN_MATCH_COUNT}")

In [ ]:
overlay_lines = []

if M is not None:
    metrics = au.calculate_projection_and_overlap(M, img1_gray.shape, img2_gray.shape)
    
    overlay_lines = [
        f"Projected area: {metrics['area_proj_px']:.1f}px^2",
        f"Overlap (on-canvas): {metrics['inter_area_px']:.1f}px^2",
        f"Overlap/img2: {metrics['rel_to_img2']*100:.2f}%",
        f"Overlap/proj: {metrics['rel_on_canvas_of_proj']*100:.2f}%",
    ]

In [ ]:
if dst is not None:
    img_result, img_matches = au.visualize_results(
        img1_color, kp1, img2_color, kp2, good_matches, 
        mask, dst, overlay_lines
    )

    # Display the result
    plt.figure(figsize=(15, 10))
    plt.imshow(cv.cvtColor(img_result, cv.COLOR_BGR2RGB))
    plt.title('Area Detection Result')
    plt.axis('off')
    plt.show()
    
    plt.figure(figsize=(20, 10))
    plt.imshow(cv.cvtColor(img_matches, cv.COLOR_BGR2RGB))
    plt.title('Feature Matches (Inliers)')
    plt.axis('off')
    plt.show()
else:
    print("No valid transformation found to visualize.")